### Importing libraries

In [98]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

### Loading datasets

In [99]:
dataset = load_dataset('imdb')
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

Found cached dataset imdb (C:/Users/Pawel/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
100%|██████████| 3/3 [00:00<00:00, 107.12it/s]


## Processing data

### Shrinking datasets

In [100]:
dataset['train'] = dataset['train'].select(range(0, 100))
dataset['test'] = dataset['test'].select(range(0, 100))
dataset['unsupervised'] = dataset['unsupervised'].select(range(0, 100))
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 100
    })
})

### Tokenizing data

In [101]:
def tokenize_function(examples):
  # extract text
  text = examples["text"]
  
  # Tokenize and truncate text
  tokenizer.truncation_side = "left"
  tokenized_inputs = tokenizer(
    text,
    return_tensors="np",
    truncation=True,
    max_length=512,
  )
  
  return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\Pawel\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-695ab50fe08d92db.arrow
Loading cached processed dataset at C:\Users\Pawel\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-12f0129256ded0ad.arrow
Loading cached processed dataset at C:\Users\Pawel\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-1b7b5ddbffa11ce1.arrow


### Transforming datasets into keras format

In [102]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
  columns=["attention_mask", "input_ids"],
  label_cols=["labels"],
  shuffle=True,
  collate_fn=data_collator,
  batch_size=8,
)

tf_validation_dataset = tokenized_dataset['test'].to_tf_dataset(
  columns=["attention_mask", "input_ids"],
  label_cols=["labels"],
  shuffle=False,
  collate_fn=data_collator,
  batch_size=8,
)

c:\Users\Pawel\anaconda3\envs\tensorflow_gpu\lib\site-packages\datasets\arrow_dataset.py:388: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


## Creating a model

In [103]:

from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

### Testing untrained model

In [105]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
  
  # tokenize text
  inputs = tokenizer.encode(text, return_tensors="pt")
  
  # # compute logits
  logits = model.predict(np.array(inputs[0]))['logits'][0]
  
  # convert logits to label
  predictions = np.argmax(logits)
  
  # print(logits)
  print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
1/1 [==============================] - 0s 169ms/step
It was good. - Negative
1/1 [==============================] - 0s 168ms/step
Not a fan, don't recommed. - Negative
1/1 [==============================] - 0s 129ms/step
Better than the first one. - Negative
1/1 [==============================] - 0s 148ms/step
This is not worth watching even once. - Negative
1/1 [==============================] - 0s 148ms/step
This one is a pass. - Negative
